# 1.Clone fairseq-py and install latest version, other packages for downloading youtube video


In [ ]:
 !pip install pytube moviepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
!mkdir "temp_dir"
!git clone https://github.com/pytorch/fairseq

# Change current working directory
!pwd
%cd "/content/fairseq"
!pip install --editable ./ 
!pip install tensorboardX


Cloning into 'fairseq'...
remote: Enumerating objects: 34719, done.
remote: Counting objects: 100% (176/176), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 34719 (delta 87), reused 142 (delta 68), pack-reused 34543
Receiving objects: 100% (34719/34719), 25.00 MiB | 26.64 MiB/s, done.
Resolving deltas: 100% (25179/25179), done.
/content
/content/fairseq
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/fairseq
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.7/273.7 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 2.Download MMS model

Un-comment to download your preferred model. In this video, I use MMS-1B-all for better model quality and language coverage, (but it would require more RAM, so please use Colab-Pro instead of Colab-Free). You can use Colab-Free for MMS-1B:FL10 or MMS-1B:L1107

In [ ]:
# MMS-1B:FL102 model - 102 Languages - FLEURS Dataset
#!wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt'

# # MMS-1B:L1107 - 1107 Languages - MMS-lab Dataset
# !wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_l1107.pt'

# # MMS-1B-all - 1162 Languages - MMS-lab + FLEURS + CV + VP + MLS
!wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_all.pt'

--2023-06-03 19:21:12--  https://dl.fbaipublicfiles.com/mms/asr/mms1b_all.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 108.157.162.120, 108.157.162.108, 108.157.162.35, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|108.157.162.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14660831159 (14G) [binary/octet-stream]
Saving to: ‘./models_new/mms1b_all.pt’

mms1b_all.pt        100%[===================>]  13.65G   156MB/s    in 93s     

2023-06-03 19:22:45 (150 MB/s) - ‘./models_new/mms1b_all.pt’ saved [14660831159/14660831159]



# 3.Download youtube CNN video

In [ ]:
from pytube import YouTube
from moviepy.editor import *
import os

# replace 'your_youtube_url_here' with your YouTube video's URL
youtube_url = 'https://www.youtube.com/watch?v=3dRQ9JApJbs'  
os.makedirs('/content/audio_chunks2', exist_ok=True)

# download video
youtube = YouTube(youtube_url)
video = youtube.streams.first()
filename = video.download()

# convert video to audio
video_clip = VideoFileClip(filename)
audio_filename = filename.split('.')[0] + '.wav'
video_clip.audio.write_audiofile(audio_filename)

# split audio into 1-minute chunks
audio_clip = AudioFileClip(audio_filename)
duration = int(audio_clip.duration)

# Initialize a counter
counter = 0

for i in range(0, duration, 60):  # we'll split the audio every 60 seconds
    start = i
    end = i + 60 if i + 60 < duration else duration  # make sure not to go over the video's duration
    subclip = audio_clip.subclip(start, end)
    # Include the counter in the filename
    subclip_filename = f"/content/audio_chunks2/{os.path.basename(audio_filename).split('.')[0]}_{str(counter).zfill(5)}.wav"
    subclip.write_audiofile(subclip_filename)
    
    # Increment the counter
    counter += 1


MoviePy - Writing audio in /content/fairseq/Unprecedented images show sunspot larger than Earth.wav


MoviePy - Done.
MoviePy - Writing audio in /content/audio_chunks2/Unprecedented images show sunspot larger than Earth_00000.wav


MoviePy - Done.
MoviePy - Writing audio in /content/audio_chunks2/Unprecedented images show sunspot larger than Earth_00001.wav


MoviePy - Done.
MoviePy - Writing audio in /content/audio_chunks2/Unprecedented images show sunspot larger than Earth_00002.wav


MoviePy - Done.
MoviePy - Writing audio in /content/audio_chunks2/Unprecedented images show sunspot larger than Earth_00003.wav


MoviePy - Done.
MoviePy - Writing audio in /content/audio_chunks2/Unprecedented images show sunspot larger than Earth_00004.wav


MoviePy - Done.


In [ ]:
# to keep the order sme
import os
import glob
import subprocess

# Define your directory
directory = '/content/audio_chunks2/'

# Use glob to get all the .wav files in the directory
input_files = glob.glob(os.path.join(directory, '*.wav'))

# Sort the files before processing
input_files = sorted(input_files)

# Loop through the input files
for i, input_filename in enumerate(input_files):
    # Define the output filename using a shorter name
    output_filename = os.path.join(directory, f'res_{i}.wav')

    # Run the ffmpeg command to resample and save with the new filename
    # convert to 16000 hz sample rate
    subprocess.run(['ffmpeg', '-y', '-i', input_filename, '-ar', '16000', output_filename])


# 4.Run Inference and transcribe the audio(s)

In [ ]:

import os
import subprocess

os.environ["TMPDIR"] = '/content/temp_dir'
os.environ["PYTHONPATH"] = "."
os.environ["PREFIX"] = "INFER"
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["USER"] = "micro"

# Define your directory and the base file name
directory = '/content/audio_chunks2/'
base_filename = 'res_'

# Define the range of file indices
file_indices = range(5)  # This will give you the numbers 0 to 5

# Initialize an empty string to hold the full transcription
full_transcription = ""

# Loop through the file indices
for i in file_indices:
    # Construct the input file name
    input_filename = os.path.join(directory, f'{base_filename}{i}.wav')

    # Define the output filename for the transcription
    transcription_filename = f'transcription_{i}.txt'

    # Open the transcription file in write mode
    with open(transcription_filename, 'w') as f:
        # Run the model inference and save the transcription to the file        # Replace mms1b_all.pt to mms1b_fl102.pt if you want to use a different model
                                                                                #https://dl.fbaipublicfiles.com/mms/asr/mms1b_all_langs.html, if you use different language
        subprocess.run(['python', 'examples/mms/asr/infer/mms_infer.py', '--model', '/content/fairseq/models_new/mmps1b_all.pt', '--lang', 'eng', '--audio', input_filename], stdout=f)

    # Open the transcription file in read mode and append its contents to the full transcription
    with open(transcription_filename, 'r') as f:
        transcription = f.read()
        full_transcription += " " + transcription

# Save the full transcription to a file in the specified directory
with open('/content/audio_chunks2/full_transcription.txt', 'w') as f:
    f.write(full_transcription)